In [1]:
import pandas as pd
import pyodbc

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
connection = pyodbc.connect(
  r'Driver={ODBC Driver 17 for SQL Server};'
  r'Server=localhost;'
  r'Database=AdventureWorks2022;'
  r'Trusted_Connection=yes;'
)

In [3]:
#class for reading table
class db_reader:
    def read_table(self, table_name, connection, col=None):
        if col:
            query = f"SELECT {', '.join(col)} FROM {table_name}" #', '.join(col) converts the list of columns into a comma-separated string. For example, if col = ['name', 'salary'], ', '.join(col) produces 'name, salary'.
        else:
            query = f"SELECT * FROM {table_name}"
        return pd.read_sql_query(query, connection)

In [4]:
# {', '.join(col)} how it works

col = ['BusinessEntityID','PersonType']

In [5]:
col = ['BusinessEntityID','PersonType']
s = ', '.join(col)

In [6]:
s

'BusinessEntityID, PersonType'

In [7]:
#class for merging
class merge:
  def merge_tbl(self, t1, t2, join, col = None, col1 = None, col2 = None):
    if col1 and col2:
      return pd.merge(t1, t2, how = join, left_on = col1, right_on = col2)
    elif col:
      return pd.merge(t1, t2, how = join, on = col)

In [8]:
# 201 Create a new table "HighValueCustomers" containing customers who have placed orders with a total due amount greater than the average total due amount

soh = db_reader().read_table('sales.salesorderheader', connection)

In [9]:
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [10]:
HighValueOrders = soh.loc[soh['TotalDue'] > 1000, ['SalesOrderID','TotalDue']]

In [11]:
HighValueOrders.head()

,SalesOrderID,TotalDue
0,43660,1457.3288
1,43661,36865.8012
2,43662,32474.9324
4,43664,27510.4109
5,43665,16158.6961


In [12]:
#202 Create a new table "NewProductSales" containing products sold within the last month, along with their sales details

pro = db_reader().read_table('production.product', connection)

In [13]:
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [14]:
sod = db_reader().read_table('sales.salesorderdetail', connection)

In [15]:
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [16]:
sale = merge().merge_tbl(pro[['ProductID', 'Name']], sod[['SalesOrderID','ProductID']], 'inner', 'ProductID')
sale.head()

,ProductID,Name,SalesOrderID
0,707,"Sport-100 Helmet, Red",43665
1,707,"Sport-100 Helmet, Red",43668
2,707,"Sport-100 Helmet, Red",43673
3,707,"Sport-100 Helmet, Red",43677
4,707,"Sport-100 Helmet, Red",43678


In [17]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [18]:
s = merge().merge_tbl(sale, soh[['SalesOrderID','OrderDate']], 'inner', 'SalesOrderID')
s.head()

,ProductID,Name,SalesOrderID,OrderDate
0,707,"Sport-100 Helmet, Red",43665,2011-05-31
1,707,"Sport-100 Helmet, Red",43668,2011-05-31
2,707,"Sport-100 Helmet, Red",43673,2011-05-31
3,707,"Sport-100 Helmet, Red",43677,2011-05-31
4,707,"Sport-100 Helmet, Red",43678,2011-05-31


In [19]:
last_mon = max(s['OrderDate'])-pd.DateOffset(months = 1)
last_mon

Timestamp('2014-05-30 00:00:00')

In [20]:
s[s['OrderDate'] > last_mon].head()

,ProductID,Name,SalesOrderID,OrderDate
3003,707,"Sport-100 Helmet, Red",74157,2014-05-31
3004,707,"Sport-100 Helmet, Red",74158,2014-05-31
3005,707,"Sport-100 Helmet, Red",74171,2014-05-31
3006,707,"Sport-100 Helmet, Red",74193,2014-06-01
3007,707,"Sport-100 Helmet, Red",74196,2014-06-01


In [21]:
#203 Create a new table "TopEmployees" containing the top 10 employees with the highest total sales amount

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [22]:
TopEmployees = soh.groupby('SalesPersonID')['TotalDue'].sum().rename('Total').reset_index().sort_values('Total', ascending = False).head(10)
TopEmployees

,SalesPersonID,Total
2,276.0,1.169502e+07
3,277.0,1.134239e+07
1,275.0,1.047537e+07
15,289.0,9.585125e+06
5,279.0,8.062920e+06
7,281.0,7.259568e+06
8,282.0,6.683537e+06
16,290.0,5.087977e+06
9,283.0,4.231048e+06
4,278.0,4.069422e+06


In [23]:
#204 Create a new table "PopularCategories" containing product categories with more than 100 units sold

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [24]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [25]:
p = merge().merge_tbl(pro[['ProductID','ProductSubcategoryID']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
p.head()

,ProductID,ProductSubcategoryID,ProductCategoryID
0,680,14.0,2
1,706,14.0,2
2,707,31.0,4
3,708,31.0,4
4,709,23.0,3


In [26]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [27]:
cat = merge().merge_tbl(p[['ProductID', 'ProductCategoryID']], sod[['ProductID', 'OrderQty']], 'inner', 'ProductID')
cat.head()

,ProductID,ProductCategoryID,OrderQty
0,707,4,1
1,707,4,2
2,707,4,4
3,707,4,1
4,707,4,1


In [28]:
PopularCategories = cat.groupby('ProductCategoryID')['OrderQty'].sum().rename('No_of_QTY_sold').reset_index().sort_values('No_of_QTY_sold', ascending=False)
PopularCategories

,ProductCategoryID,No_of_QTY_sold
0,1,90258
2,3,73658
3,4,61928
1,2,49044


In [29]:
#205 Create a new table "HighValueOrders" containing orders with a total due amount greater than $5000

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [30]:
HighValueOrders = soh.loc[soh['TotalDue'] > 5000, ['SalesOrderID','TotalDue']]

In [31]:
HighValueOrders.head()

,SalesOrderID,TotalDue
1,43661,36865.8012
2,43662,32474.9324
4,43664,27510.4109
5,43665,16158.6961
6,43666,5694.8564


In [32]:
#206 Retrieve the list of products with a price higher than the average price of products in their respective categories

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [33]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [34]:
cat= merge().merge_tbl(pro[['ProductID','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID','ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [35]:
price = cat[cat['ListPrice'] > cat['ListPrice'].mean()]
price.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.5,2
1,706,14.0,1431.5,2
12,717,14.0,1431.5,2
13,718,14.0,1431.5,2
14,719,14.0,1431.5,2


In [36]:
result = cat[cat['ProductID'].isin(price['ProductID'])]

In [37]:
result.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.5,2
1,706,14.0,1431.5,2
12,717,14.0,1431.5,2
13,718,14.0,1431.5,2
14,719,14.0,1431.5,2


In [38]:
#207 Get the list of employees whose total sales amount exceeds the average total sales amount of employees in their respective departments

dep = db_reader().read_table('humanresources.department', connection)
dep.head()


,DepartmentID,Name,GroupName,ModifiedDate
0,1,Engineering,Research and Development,2008-04-30
1,2,Tool Design,Research and Development,2008-04-30
2,3,Sales,Sales and Marketing,2008-04-30
3,4,Marketing,Sales and Marketing,2008-04-30
4,5,Purchasing,Inventory Management,2008-04-30


In [39]:
edh = db_reader().read_table('humanresources.employeedepartmenthistory', connection)
edh.head()

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13
1,2,1,1,2008-01-31,None,2008-01-30
2,3,1,1,2007-11-11,None,2007-11-10
3,4,1,1,2007-12-05,2010-05-30,2010-05-28
4,4,2,1,2010-05-31,None,2010-05-30


In [40]:
d = merge().merge_tbl(dep[['DepartmentID']], edh[['BusinessEntityID', 'DepartmentID']], 'inner', 'DepartmentID')
d.head()

,DepartmentID,BusinessEntityID
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6


In [41]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [42]:
de = merge().merge_tbl(d, soh[['SalesPersonID', 'TotalDue']], 'inner', col1 = 'BusinessEntityID', col2 = 'SalesPersonID')
de.head()

,DepartmentID,BusinessEntityID,SalesPersonID,TotalDue
0,3,274,274.0,23130.2957
1,3,274,274.0,2297.0332
2,3,274,274.0,4723.1073
3,3,274,274.0,2417.4793
4,3,274,274.0,68918.2404


In [43]:
de.groupby(['DepartmentID','BusinessEntityID'])['TotalDue'].sum().rename('TotalSales').reset_index().query('TotalSales > TotalSales.mean()')

,DepartmentID,BusinessEntityID,TotalSales
1,3,275,1.047537e+07
2,3,276,1.169502e+07
3,3,277,1.134239e+07
5,3,279,8.062920e+06
7,3,281,7.259568e+06
8,3,282,6.683537e+06
15,3,289,9.585125e+06


In [44]:
#208 Retrieve orders where the total due amount is greater than the average total due amount of orders placed by customers in their respective territories

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [45]:
soh.groupby(['TerritoryID', 'CustomerID'])['TotalDue'].sum().rename('Sales').reset_index().query('Sales > Sales.mean()').tail(10)

,TerritoryID,CustomerID,Sales
19104,10,29941,176701.5844
19105,10,29952,8669.5054
19108,10,29985,70217.2163
19110,10,30005,109753.0267
19111,10,30019,24931.8214
19113,10,30030,68644.5658
19114,10,30044,159695.4590
19115,10,30050,631355.0743
19117,10,30102,7095.0672
19119,10,30113,307946.0796


In [46]:
#209 Get customers who have placed orders in territories where the total number of customers exceeds the average number of customers across all territories

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [47]:
soh.groupby(['TerritoryID', 'CustomerID'])['CustomerID'].count().rename('count').reset_index().query('count > count.mean()').head(10)


,TerritoryID,CustomerID,count
0,1,11012,2
1,1,11013,2
2,1,11014,2
8,1,11066,2
9,1,11067,2
10,1,11084,2
11,1,11128,2
12,1,11129,2
15,1,11137,2
19,1,11153,2


In [48]:
#210 Retrieve products from categories where the average list price is greater than the average list price of all categories

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [49]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [50]:
cat= merge().merge_tbl(pro[['ProductID','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID','ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [51]:
cat.groupby(['ProductID', 'ProductCategoryID'])['ListPrice'].mean().rename('avg').reset_index().query('avg < @cat["ListPrice"].mean()').head()

,ProductID,ProductCategoryID,avg
2,707,4,34.99
3,708,4,34.99
4,709,3,9.50
5,710,3,9.50
6,711,4,34.99


In [52]:
#211 Get the total sales amount for each product category, along with the number of products in each category

pro = db_reader().read_table('production.product', connection)
pro.head()


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [53]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [54]:
cat= merge().merge_tbl(pro[['ProductID','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID','ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [55]:
cat.groupby('ProductCategoryID').agg({'ListPrice' : 'sum', 'ProductID' : 'count'}).rename(columns={'ListPrice' : 'Sum', 'ProductID' : 'Count'}).reset_index().head()

,ProductCategoryID,Sum,Count
0,1,14227.378,97
1,2,62961.280,134
2,3,1793.600,36
3,4,996.120,29


In [56]:
#212 Retrieve the total number of orders placed by each customer in each year

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [57]:
soh.groupby(soh['OrderDate'].dt.year)['SalesOrderID'].count().rename('count').reset_index().head()

,OrderDate,count
0,2011,1607
1,2012,3915
2,2013,14182
3,2014,11761


In [58]:
# 213 Get the total sales amount for each employee in each year

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [59]:
soh.groupby([soh['OrderDate'].dt.year, 'SalesPersonID'])['TotalDue'].sum().rename('count').reset_index().tail()

,OrderDate,SalesPersonID,count
53,2014,286.0,6.595481e+05
54,2014,287.0,6.374784e+04
55,2014,288.0,6.548992e+05
56,2014,289.0,1.558542e+06
57,2014,290.0,9.777248e+05


In [60]:
# 214 Retrieve the total number of products sold in each territory

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [61]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [62]:
s = merge().merge_tbl(soh[['SalesOrderID', 'TotalDue', 'TerritoryID']], sod[['SalesOrderID']], 'inner', 'SalesOrderID')
s.head()

,SalesOrderID,TotalDue,TerritoryID
0,43660,1457.3288,5
1,43660,1457.3288,5
2,43661,36865.8012,6
3,43661,36865.8012,6
4,43661,36865.8012,6


In [63]:
s.groupby('TerritoryID')['SalesOrderID'].count().rename('count').reset_index().tail()

,TerritoryID,count
5,6,19064
6,7,9088
7,8,7528
8,9,15058
9,10,10426


In [64]:
# 215 Get the total sales amount for each month of the year

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [65]:
soh.groupby([soh['OrderDate'].dt.month.rename('Month'),soh['OrderDate'].dt.year.rename('Year')])['TotalDue'].sum().rename('count').reset_index().head()

,Month,Year,count
0,1,2012,4.458337e+06
1,1,2013,2.340062e+06
2,1,2014,4.798028e+06
3,2,2012,1.649052e+06
4,2,2013,2.600219e+06


In [66]:
# 216 Retrieve product categories with an average list price greater than the average list price of all categories and with at least 10 products

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [67]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [68]:
cat= merge().merge_tbl(pro[['ProductID','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID','ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [69]:
cat.groupby(['ProductCategoryID']).agg({'ProductID' : 'count', 'ListPrice' : 'mean'}).rename(columns={'ProductID' : 'count', 'ListPrice' : 'mean'}).reset_index().head().query('mean > mean.mean()')

,ProductCategoryID,count,mean
1,2,134,469.860299


In [70]:
cat.groupby(['ProductID', 'ProductCategoryID']).agg({'ListPrice' : 'mean', 'ProductID': 'count'}).rename(columns= {'ListPrice' : 'avg', 'ProductID' : 'count'}).reset_index().query('avg < @cat["ListPrice"].mean() and count >=10').head()

,ProductID,ProductCategoryID,avg,count


In [71]:
# 217 Get employees with a total sales amount greater than $50000 and who have made sales to at least 10 different customers

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [72]:
soh.groupby('SalesPersonID')['CustomerID'].nunique().reset_index(name='count').query('count >= 10') .head()

,SalesPersonID,count
0,274.0,44
1,275.0,118
2,276.0,69
3,277.0,121
4,278.0,38


In [73]:
# 218 Retrieve territories with more than 5 customers and with a total sales amount greater than $100000

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [74]:
soh.groupby('TerritoryID').agg({'CustomerID' : 'nunique', 'TotalDue' : 'sum'}).rename(columns = {'CustomerID': 'number', 'TotalDue' : 'Total'}).reset_index().query('Total > 100000').head()

,TerritoryID,number,Total
0,1,3428,1.806166e+07
1,2,57,7.820210e+06
2,3,69,8.913299e+06
3,4,4565,2.715059e+07
4,5,92,8.884099e+06


In [75]:
# 219 Get product categories with an average list price greater than $500 and with at least 20 units sold

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [76]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [77]:
a = merge().merge_tbl(pro[['ProductID','ListPrice','ProductSubcategoryID']], sod[['ProductID', 'OrderQty']], 'inner', 'ProductID')
a.head()

,ProductID,ListPrice,ProductSubcategoryID,OrderQty
0,707,34.99,31.0,1
1,707,34.99,31.0,2
2,707,34.99,31.0,4
3,707,34.99,31.0,1
4,707,34.99,31.0,1


In [78]:
pro = db_reader().read_table('production.productsubcategory', connection)
pro.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [79]:
prod = merge().merge_tbl(a, pro[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
prod.head()

,ProductID,ListPrice,ProductSubcategoryID,OrderQty,ProductCategoryID
0,707,34.99,31.0,1,4
1,707,34.99,31.0,2,4
2,707,34.99,31.0,4,4
3,707,34.99,31.0,1,4
4,707,34.99,31.0,1,4


In [80]:
prod.groupby('ProductCategoryID').agg({'ListPrice' : 'mean', 'OrderQty' : 'sum'}).rename(columns = {'ListPrice' : 'avg', 'OrderQty' : 'sum'}).reset_index().query('avg > 500').head()


,ProductCategoryID,avg,sum


In [81]:
# 220 Retrieve orders with a total due amount greater than $5000 and with at least 3 different products

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [82]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [83]:
result = merge().merge_tbl(sod[['SalesOrderID', 'ProductID']], soh[['SalesOrderID', 'TotalDue']], 'inner', 'SalesOrderID')
result.head()

,SalesOrderID,ProductID,TotalDue
0,43660,762,1457.3288
1,43660,758,1457.3288
2,43661,745,36865.8012
3,43661,743,36865.8012
4,43661,747,36865.8012


In [84]:
result.groupby('SalesOrderID').agg({'ProductID' : 'nunique', 'TotalDue' : 'sum'}).rename(columns = {'ProductID' : 'count', 'TotalDue' : 'sum'}).reset_index().query('count > 3 and sum > 5000').head()

,SalesOrderID,count,sum
1,43661,15,552987.0180
2,43662,22,714448.5128
4,43664,8,220083.2872
5,43665,10,161586.9610
6,43666,6,34169.1384


In [85]:
# 221 Retrieve the top 5 customers with the highest total order amounts, excluding customers who have placed less than 3 orders

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [86]:
soh.groupby('CustomerID').agg({'SalesOrderID': 'count', 'TotalDue': 'sum'}).rename(columns={'SalesOrderID': 'count', 'TotalDue': 'sum'}).reset_index().query('count <= 3').sort_values('sum', ascending=False).head() 


,CustomerID,count,sum
18600,29600,3,19798.5095
19013,30013,3,16897.7220
18808,29808,3,12665.7725
18623,29623,3,12019.4344
18948,29948,3,11325.5993


In [87]:
#222 Get the top 10 products with the highest profit margins, calculated as the difference between the list price and the standard cost, excluding products with a profit margin less than $50

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [88]:
pro['PriceDifference'] = pro['ListPrice'] - pro['StandardCost']

In [89]:
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate,PriceDifference
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827,1000.0
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827,0.0
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827,0.0
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827,0.0
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827,0.0


In [90]:
filtered_pro = pro[pro['PriceDifference'] > 50]

In [91]:
result = filtered_pro.sort_values(by='PriceDifference', ascending=False).head(10)
result

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate,PriceDifference
505,2006,BIKE HANDLE,RV-H123,True,True,BLUE,250,20,145.8800,2000.0,20,CM,LB,2.00,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,D67538F2-A81A-4A4F-98F7-202002D888B4,2024-03-02 14:57:55.763,1854.1200
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827,1000.0000
244,740,"HL Mountain Frame - Silver, 44",FR-M94S-44,True,True,Silver,500,375,706.8110,1364.5,44,CM,LB,2.76,1,M,H,U,12.0,5.0,2011-05-31,2012-05-29,NaT,1909C60C-C490-411D-B3E6-12DDD7832482,2014-02-08 10:01:36.827,657.6890
245,741,"HL Mountain Frame - Silver, 48",FR-M94S-52,True,True,Silver,500,375,706.8110,1364.5,48,CM,LB,2.80,1,M,H,U,12.0,5.0,2011-05-31,2012-05-29,NaT,B181EC1F-CA20-4724-B2EB-15F3E455142E,2014-02-08 10:01:36.827,657.6890
249,745,"HL Mountain Frame - Black, 48",FR-M94B-48,True,True,Black,500,375,699.0928,1349.6,48,CM,LB,2.80,1,M,H,U,12.0,5.0,2011-05-31,2012-05-29,NaT,1FEE0573-6676-432D-8D6D-41BA9FAA5865,2014-02-08 10:01:36.827,650.5072
248,744,"HL Mountain Frame - Black, 44",FR-M94B-44,True,True,Black,500,375,699.0928,1349.6,44,CM,LB,2.76,1,M,H,U,12.0,5.0,2011-05-31,2012-05-29,NaT,CB443286-6B25-409F-A10B-1AD4EEB4BD4E,2014-02-08 10:01:36.827,650.5072
252,748,"HL Mountain Frame - Silver, 38",FR-M94S-38,True,True,Silver,500,375,747.2002,1364.5,38,CM,LB,2.68,2,M,H,U,12.0,5.0,2011-05-31,NaT,NaT,F246ACAA-A80B-40EC-9208-02EDEF885129,2014-02-08 10:01:36.827,617.2998
243,739,"HL Mountain Frame - Silver, 42",FR-M94S-42,True,True,Silver,500,375,747.2002,1364.5,42,CM,LB,2.72,1,M,H,U,12.0,5.0,2011-05-31,NaT,NaT,8AE32663-8D6F-457D-8343-5B181FEC43A7,2014-02-08 10:01:36.827,617.2998
246,742,"HL Mountain Frame - Silver, 46",FR-M94S-46,True,True,Silver,500,375,747.2002,1364.5,46,CM,LB,2.84,1,M,H,U,12.0,5.0,2011-05-31,NaT,NaT,A189D86E-D923-4336-B13D-A5DB6F426540,2014-02-08 10:01:36.827,617.2998
251,747,"HL Mountain Frame - Black, 38",FR-M94B-38,True,True,Black,500,375,739.0410,1349.6,38,CM,LB,2.68,2,M,H,U,12.0,5.0,2011-05-31,NaT,NaT,0C548577-3171-4CE2-B9A0-1ED526849DE8,2014-02-08 10:01:36.827,610.5590


In [92]:
#223 Retrieve the top 3 employees with the highest average sales amount per order, excluding employees with less than 10 orders

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [93]:
soh.groupby('SalesPersonID').agg({'TotalDue':'mean', 'SalesPersonID' : 'count'}).rename(columns = {'TotalDue':'avg', 'SalesPersonID' : 'count'}).reset_index().query('count >10').sort_values('avg', ascending=False).head(3)

,SalesPersonID,avg,count
6,280.0,39455.222335,95
7,281.0,29998.214364,242
16,290.0,29074.155497,175


In [94]:
#224 Get the top 5 product categories with the highest average list price, excluding categories with less than 10 products

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [95]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [96]:
cat = merge().merge_tbl(p[['ProductID','ProductSubcategoryID','ListPrice']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [97]:
cat.groupby('ProductCategoryID').agg({'ProductID' : 'count', 'ListPrice' : 'mean'}).rename(columns={'ProductID' : 'count', 'ListPrice' : 'avg'}).reset_index().query('count > 10').head()

,ProductCategoryID,count,avg
0,1,97,146.674000
1,2,134,469.860299
2,3,36,49.822222
3,4,29,34.348966


In [98]:
#225 Retrieve the top 10 orders with the highest total due amount, excluding orders with less than 5 products

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [99]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [100]:
pro = merge().merge_tbl(soh[['SalesOrderID', 'TotalDue']],sod[['SalesOrderID', 'ProductID']], 'inner', 'SalesOrderID')
pro.head()

,SalesOrderID,TotalDue,ProductID
0,43660,1457.3288,762
1,43660,1457.3288,758
2,43661,36865.8012,745
3,43661,36865.8012,743
4,43661,36865.8012,747


In [101]:
pro.groupby('SalesOrderID').agg({'TotalDue' : 'max', 'ProductID' : 'count'}).rename(columns = {'TotalDue' : 'Total', 'ProductID' : 'NoOfProducts'}).reset_index().query('NoOfProducts > 5').sort_values('Total', ascending=False).head()

,SalesOrderID,Total,NoOfProducts
7471,51131,187487.8250,60
11622,55282,182018.6272,51
2956,46616,170512.6689,65
3321,46981,166537.0808,65
3735,47395,165028.7482,66


In [102]:
#226 Retrieve the first and last names of employees along with their corresponding job titles.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

In [103]:
emp = db_reader().read_table('humanresources.employee',connection,col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [104]:
per = db_reader().read_table('person.person',connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [105]:
name = merge().merge_tbl(emp[['BusinessEntityID', 'JobTitle']], per[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', 'BusinessEntityID')
name.head()

,BusinessEntityID,JobTitle,FirstName,LastName
0,1,Chief Executive Officer,Ken,Sánchez
1,2,Vice President of Engineering,Terri,Duffy
2,3,Engineering Manager,Roberto,Tamburello
3,4,Senior Tool Designer,Rob,Walters
4,5,Design Engineer,Gail,Erickson


In [106]:
#227 Get the list of products along with their corresponding product categories, including products without a category

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [107]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [108]:
cat = merge().merge_tbl(p[['ProductID','ProductSubcategoryID','ListPrice']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'outer', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,771.0,1.0,146.674,1.0
1,772.0,1.0,146.674,1.0
2,773.0,1.0,146.674,1.0
3,774.0,1.0,146.674,1.0
4,775.0,1.0,146.674,1.0


In [109]:
cat[['ProductCategoryID', 'ProductID']].tail()

,ProductCategoryID,ProductID
502,NaN,533.0
503,NaN,534.0
504,NaN,535.0
505,NaN,679.0
506,NaN,2006.0


In [110]:
#228 Retrieve the list of product categories along with the names of products belonging to each category, including categories without any products.

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [111]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [112]:
cat = merge().merge_tbl(p[['ProductID','ProductSubcategoryID','Name']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'outer', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,Name,ProductCategoryID
0,771.0,1.0,"Mountain-100 Silver, 38",1.0
1,772.0,1.0,"Mountain-100 Silver, 42",1.0
2,773.0,1.0,"Mountain-100 Silver, 44",1.0
3,774.0,1.0,"Mountain-100 Silver, 48",1.0
4,775.0,1.0,"Mountain-100 Black, 38",1.0


In [113]:
cat[['ProductID', 'ProductCategoryID', 'Name']].head()

,ProductID,ProductCategoryID,Name
0,771.0,1.0,"Mountain-100 Silver, 38"
1,772.0,1.0,"Mountain-100 Silver, 42"
2,773.0,1.0,"Mountain-100 Silver, 44"
3,774.0,1.0,"Mountain-100 Silver, 48"
4,775.0,1.0,"Mountain-100 Black, 38"


In [114]:
#229 Retrieve a list of customers and orders, including customers who haven't placed any orders and orders without associated customers.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [115]:
cus = db_reader().read_table('sales.customer', connection)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263


In [116]:
res = merge().merge_tbl(soh[['SalesOrderID', 'CustomerID']], cus[['PersonID']], 'outer', col1 = 'CustomerID', col2 = 'PersonID')
res.head()

,SalesOrderID,CustomerID,PersonID
0,NaN,NaN,291.0
1,NaN,NaN,293.0
2,NaN,NaN,295.0
3,NaN,NaN,297.0
4,NaN,NaN,299.0


In [117]:
#230 Generate a Cartesian product of product categories and territories, showing all possible combinations.

cat = db_reader().read_table('production.productcategory', connection)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [118]:
t = db_reader().read_table('sales.salesterritory', connection)
t.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [119]:
cart = pd.merge(cat[['ProductCategoryID']],t[['TerritoryID']], how='cross')
cart.head()

,ProductCategoryID,TerritoryID
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


In [120]:
# 231 Retrieve the names of employees along with the names of their managers.

In [121]:
# 232 Retrieve the list of products and their corresponding product descriptions.

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [122]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [123]:
cat = merge().merge_tbl(p[['ProductID','ProductSubcategoryID','Name', 'ProductModelID']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'outer', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,Name,ProductModelID,ProductCategoryID
0,771.0,1.0,"Mountain-100 Silver, 38",19.0,1.0
1,772.0,1.0,"Mountain-100 Silver, 42",19.0,1.0
2,773.0,1.0,"Mountain-100 Silver, 44",19.0,1.0
3,774.0,1.0,"Mountain-100 Silver, 48",19.0,1.0
4,775.0,1.0,"Mountain-100 Black, 38",19.0,1.0


In [124]:
d = db_reader().read_table('production.ProductDescription', connection)
d.head()

,ProductDescriptionID,Description,rowguid,ModifiedDate
0,3,NEW DESCRIPTION,301EED3A-1A82-4855-99CB-2AFE8290D641,2013-04-30
1,4,Aluminum alloy cups; large diameter spindle.,DFEBA528-DA11-4650-9D86-CAFDA7294EB0,2013-04-30
2,5,Aluminum alloy cups and a hollow axle.,F7178DA7-1A7E-4997-8470-06737181305E,2013-04-30
3,8,"Suitable for any type of riding, on or off-roa...",8E6746E5-AD97-46E2-BD24-FCEA075C3B52,2013-04-30
4,64,This bike delivers a high-level of performance...,7B1C4E90-85E2-4792-B47B-E0C424E2EC94,2013-04-30


In [125]:
dc = db_reader().read_table('production.ProductModelProductDescriptionCulture', connection)
dc.head()

,ProductModelID,ProductDescriptionID,CultureID,ModifiedDate
0,1,1199,en,2013-04-30
1,1,1467,ar,2013-04-30
2,1,1589,fr,2013-04-30
3,1,1712,th,2013-04-30
4,1,1838,he,2013-04-30


In [126]:
des = merge().merge_tbl(cat[['ProductID','Name','ProductModelID']], dc[['ProductModelID', 'ProductDescriptionID']], 'inner', 'ProductModelID')
des.head()

,ProductID,Name,ProductModelID,ProductDescriptionID
0,771.0,"Mountain-100 Silver, 38",19.0,168
1,771.0,"Mountain-100 Silver, 38",19.0,1367
2,771.0,"Mountain-100 Silver, 38",19.0,1491
3,771.0,"Mountain-100 Silver, 38",19.0,1613
4,771.0,"Mountain-100 Silver, 38",19.0,1735


In [127]:
id = merge().merge_tbl(des[['ProductID','Name','ProductDescriptionID']], d[['ProductDescriptionID', 'Description']], 'inner','ProductDescriptionID')
id.head()

,ProductID,Name,ProductDescriptionID,Description
0,771.0,"Mountain-100 Silver, 38",168,Top-of-the-line competition mountain bike. Per...
1,771.0,"Mountain-100 Silver, 38",1367,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...
2,771.0,"Mountain-100 Silver, 38",1491,VTT de compétition haut de gamme. Plusieurs op...
3,771.0,"Mountain-100 Silver, 38",1613,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...
4,771.0,"Mountain-100 Silver, 38",1735,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...


In [128]:
# 233 Retrieve the list of orders along with the names of customers who placed those orders, matching on both customer ID and territory ID.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [129]:
cus = db_reader().read_table('person.person', connection)
cus.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [130]:
id = merge().merge_tbl(soh[['SalesOrderID', 'CustomerID', 'TerritoryID']], cus[['BusinessEntityID', 'FirstName' ,'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
id.head()

,SalesOrderID,CustomerID,TerritoryID,BusinessEntityID,FirstName,LastName
0,43700,14501,4,14501,Bryan,Gray
1,43701,11003,9,11003,Jenna,Young
2,43703,16624,9,16624,Nichole,Goel
3,43704,11005,9,11005,Joe,Madan
4,43705,11011,9,11011,Joe,Rana


In [131]:
# 234 Get the total sales amount for each product, joining the SalesOrderDetail table with the Product table.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [132]:
p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [133]:
sal = merge().merge_tbl(sod[['ProductID', 'LineTotal']], p[['ProductID']], 'inner', 'ProductID')
sal.tail()

,ProductID,LineTotal
121300,878,21.98
121301,712,8.99
121302,878,21.98
121303,879,159.00
121304,712,8.99


In [134]:
sal.groupby('ProductID')['LineTotal'].sum().reset_index().head()

,ProductID,LineTotal
0,707,157772.394392
1,708,160869.517836
2,709,6026.188200
3,710,513.000000
4,711,165325.871049


In [135]:
# 235 Retrieve the list of employees who have sold products with a price higher than the average price of all products.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [136]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [137]:
s = merge().merge_tbl(sod[['SalesOrderID', 'ProductID', 'LineTotal']], soh[['SalesOrderID', 'SalesPersonID']], 'inner', 'SalesOrderID')
s.head()

,SalesOrderID,ProductID,LineTotal,SalesPersonID
0,43660,762,419.4589,279.0
1,43660,758,874.7940,279.0
2,43661,745,809.7600,282.0
3,43661,743,714.7043,282.0
4,43661,747,1429.4086,282.0


In [139]:
s.groupby(['SalesPersonID', 'ProductID', 'LineTotal'])['LineTotal'].mean().rename('avg').reset_index().query('avg < LineTotal').head()

,SalesPersonID,ProductID,LineTotal,avg
27,274.0,711,104.9700,104.9700
650,275.0,707,60.5595,60.5595
656,275.0,707,104.9700,104.9700
677,275.0,708,60.5595,60.5595
710,275.0,711,60.5595,60.5595


In [140]:
# 236 Retrieve the product name and list price along with the product category name for each product.

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [141]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [142]:
cat = merge().merge_tbl(p[['Name','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,Name,ProductSubcategoryID,ListPrice,ProductCategoryID
0,"HL Road Frame - Black, 58",14.0,1431.50,2
1,"HL Road Frame - Red, 58",14.0,1431.50,2
2,"Sport-100 Helmet, Red",31.0,34.99,4
3,"Sport-100 Helmet, Black",31.0,34.99,4
4,"Mountain Bike Socks, M",23.0,9.50,3


In [143]:
pc = db_reader().read_table('production.productcategory', connection)
pc.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [144]:
c = merge().merge_tbl(cat[['Name','ProductCategoryID', 'ListPrice']], pc[['ProductCategoryID', 'Name']], 'inner', 'ProductCategoryID')
c.head()

,Name_x,ProductCategoryID,ListPrice,Name_y
0,"HL Road Frame - Black, 58",2,1431.50,Components
1,"HL Road Frame - Red, 58",2,1431.50,Components
2,"Sport-100 Helmet, Red",4,34.99,Accessories
3,"Sport-100 Helmet, Black",4,34.99,Accessories
4,"Mountain Bike Socks, M",3,9.50,Clothing


In [147]:
# 237 Get the list of employees along with their corresponding departments, including employees who are not assigned to any department.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [149]:
d = db_reader().read_table('humanresources.department', connection)
d.head()

,DepartmentID,Name,GroupName,ModifiedDate
0,1,Engineering,Research and Development,2008-04-30
1,2,Tool Design,Research and Development,2008-04-30
2,3,Sales,Sales and Marketing,2008-04-30
3,4,Marketing,Sales and Marketing,2008-04-30
4,5,Purchasing,Inventory Management,2008-04-30


In [150]:
d_id = db_reader().read_table('humanresources.employeedepartmenthistory', connection)
d_id.head()

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13
1,2,1,1,2008-01-31,None,2008-01-30
2,3,1,1,2007-11-11,None,2007-11-10
3,4,1,1,2007-12-05,2010-05-30,2010-05-28
4,4,2,1,2010-05-31,None,2010-05-30


In [151]:
dept = merge().merge_tbl(d[['DepartmentID', 'Name']], d_id[['DepartmentID', 'BusinessEntityID']], 'outer', 'DepartmentID')
dept.head()

,DepartmentID,Name,BusinessEntityID
0,1,Engineering,2
1,1,Engineering,3
2,1,Engineering,4
3,1,Engineering,5
4,1,Engineering,6


In [152]:
e = merge().merge_tbl(emp[['BusinessEntityID']], dept[['BusinessEntityID', 'Name']], 'outer', 'BusinessEntityID')
e.head()

,BusinessEntityID,Name
0,1,Executive
1,2,Engineering
2,3,Engineering
3,4,Engineering
4,4,Tool Design


In [153]:
# 238 Retrieve the list of product categories along with the names of products belonging to each category, including categories without any products.

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [154]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [159]:
cat = merge().merge_tbl(p[['Name','ProductSubcategoryID']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'outer', 'ProductSubcategoryID')
cat.head()

,Name,ProductSubcategoryID,ProductCategoryID
0,"Mountain-100 Silver, 38",1.0,1.0
1,"Mountain-100 Silver, 42",1.0,1.0
2,"Mountain-100 Silver, 44",1.0,1.0
3,"Mountain-100 Silver, 48",1.0,1.0
4,"Mountain-100 Black, 38",1.0,1.0


In [160]:
pc = db_reader().read_table('production.productcategory', connection)
pc.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [161]:
c = merge().merge_tbl(cat[['Name','ProductCategoryID']], pc[['ProductCategoryID', 'Name']], 'outer', 'ProductCategoryID')
c.head()

,Name_x,ProductCategoryID,Name_y
0,"Mountain-100 Silver, 38",1.0,Bikes
1,"Mountain-100 Silver, 42",1.0,Bikes
2,"Mountain-100 Silver, 44",1.0,Bikes
3,"Mountain-100 Silver, 48",1.0,Bikes
4,"Mountain-100 Black, 38",1.0,Bikes


In [162]:
# 239 Retrieve a list of customers and orders, including customers who haven't placed any orders and orders without associated customers.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [163]:
cus = db_reader().read_table('sales.customer', connection)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263


In [164]:
s = merge().merge_tbl(soh[['CustomerID','SalesOrderID']], cus['PersonID'], 'outer', col1 = 'CustomerID', col2 = 'PersonID')
s.head()

,CustomerID,SalesOrderID,PersonID
0,NaN,NaN,291.0
1,NaN,NaN,293.0
2,NaN,NaN,295.0
3,NaN,NaN,297.0
4,NaN,NaN,299.0


In [165]:
# 240 Generate a Cartesian product of product categories and territories, showing all possible combinations.

cat = db_reader().read_table('production.productcategory', connection)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [166]:
t = db_reader().read_table('sales.salesterritory', connection)
t.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [167]:
cart = pd.merge(cat[['ProductCategoryID']],t[['TerritoryID']], how='cross')
cart.head()

,ProductCategoryID,TerritoryID
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


In [ ]:
# 241 Retrieve the names of employees along with the names of their managers.

In [168]:
# 242 Retrieve the list of products and their corresponding product descriptions.

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [169]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [170]:
cat = merge().merge_tbl(p[['ProductID','ProductSubcategoryID','Name', 'ProductModelID']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'outer', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,Name,ProductModelID,ProductCategoryID
0,771.0,1.0,"Mountain-100 Silver, 38",19.0,1.0
1,772.0,1.0,"Mountain-100 Silver, 42",19.0,1.0
2,773.0,1.0,"Mountain-100 Silver, 44",19.0,1.0
3,774.0,1.0,"Mountain-100 Silver, 48",19.0,1.0
4,775.0,1.0,"Mountain-100 Black, 38",19.0,1.0


In [171]:
d = db_reader().read_table('production.ProductDescription', connection)
d.head()

,ProductDescriptionID,Description,rowguid,ModifiedDate
0,3,NEW DESCRIPTION,301EED3A-1A82-4855-99CB-2AFE8290D641,2013-04-30
1,4,Aluminum alloy cups; large diameter spindle.,DFEBA528-DA11-4650-9D86-CAFDA7294EB0,2013-04-30
2,5,Aluminum alloy cups and a hollow axle.,F7178DA7-1A7E-4997-8470-06737181305E,2013-04-30
3,8,"Suitable for any type of riding, on or off-roa...",8E6746E5-AD97-46E2-BD24-FCEA075C3B52,2013-04-30
4,64,This bike delivers a high-level of performance...,7B1C4E90-85E2-4792-B47B-E0C424E2EC94,2013-04-30


In [172]:
dc = db_reader().read_table('production.ProductModelProductDescriptionCulture', connection)
dc.head()

,ProductModelID,ProductDescriptionID,CultureID,ModifiedDate
0,1,1199,en,2013-04-30
1,1,1467,ar,2013-04-30
2,1,1589,fr,2013-04-30
3,1,1712,th,2013-04-30
4,1,1838,he,2013-04-30


In [173]:
des = merge().merge_tbl(cat[['ProductID','Name','ProductModelID']], dc[['ProductModelID', 'ProductDescriptionID']], 'inner', 'ProductModelID')
des.head()

,ProductID,Name,ProductModelID,ProductDescriptionID
0,771.0,"Mountain-100 Silver, 38",19.0,168
1,771.0,"Mountain-100 Silver, 38",19.0,1367
2,771.0,"Mountain-100 Silver, 38",19.0,1491
3,771.0,"Mountain-100 Silver, 38",19.0,1613
4,771.0,"Mountain-100 Silver, 38",19.0,1735


In [174]:
id = merge().merge_tbl(des[['ProductID','Name','ProductDescriptionID']], d[['ProductDescriptionID', 'Description']], 'inner','ProductDescriptionID')
id.head()

,ProductID,Name,ProductDescriptionID,Description
0,771.0,"Mountain-100 Silver, 38",168,Top-of-the-line competition mountain bike. Per...
1,771.0,"Mountain-100 Silver, 38",1367,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...
2,771.0,"Mountain-100 Silver, 38",1491,VTT de compétition haut de gamme. Plusieurs op...
3,771.0,"Mountain-100 Silver, 38",1613,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...
4,771.0,"Mountain-100 Silver, 38",1735,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...


In [175]:
# 243 Retrieve the list of orders along with the names of customers who placed those orders, matching on both customer ID and territory ID.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [176]:
cus = db_reader().read_table('person.person', connection)
cus.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [177]:
id = merge().merge_tbl(soh[['SalesOrderID', 'CustomerID', 'TerritoryID']], cus[['BusinessEntityID', 'FirstName' ,'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
id.head()

,SalesOrderID,CustomerID,TerritoryID,BusinessEntityID,FirstName,LastName
0,43700,14501,4,14501,Bryan,Gray
1,43701,11003,9,11003,Jenna,Young
2,43703,16624,9,16624,Nichole,Goel
3,43704,11005,9,11005,Joe,Madan
4,43705,11011,9,11011,Joe,Rana


In [178]:
# 244 Get the total sales amount for each product, joining the SalesOrderDetail table with the Product table.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [179]:
p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [180]:
sal = merge().merge_tbl(sod[['ProductID', 'LineTotal']], p[['ProductID']], 'inner', 'ProductID')
sal.tail()

,ProductID,LineTotal
121300,878,21.98
121301,712,8.99
121302,878,21.98
121303,879,159.00
121304,712,8.99


In [181]:
sal.groupby('ProductID')['LineTotal'].sum().reset_index().head()

,ProductID,LineTotal
0,707,157772.394392
1,708,160869.517836
2,709,6026.188200
3,710,513.000000
4,711,165325.871049


In [182]:
# 245 Retrieve the list of employees who have sold products with a price higher than the average price of all products.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [183]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [184]:
s = merge().merge_tbl(sod[['SalesOrderID', 'ProductID', 'LineTotal']], soh[['SalesOrderID', 'SalesPersonID']], 'inner', 'SalesOrderID')
s.head()

,SalesOrderID,ProductID,LineTotal,SalesPersonID
0,43660,762,419.4589,279.0
1,43660,758,874.7940,279.0
2,43661,745,809.7600,282.0
3,43661,743,714.7043,282.0
4,43661,747,1429.4086,282.0


In [185]:
s.groupby(['SalesPersonID', 'ProductID', 'LineTotal'])['LineTotal'].mean().rename('avg').reset_index().query('avg < LineTotal').head()

,SalesPersonID,ProductID,LineTotal,avg
27,274.0,711,104.9700,104.9700
650,275.0,707,60.5595,60.5595
656,275.0,707,104.9700,104.9700
677,275.0,708,60.5595,60.5595
710,275.0,711,60.5595,60.5595


In [186]:
# 246 Retrieve the product name, list price, and color along with the product category name for each product, using aliases for table names.

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [187]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [188]:
cat = merge().merge_tbl(p[['Name','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()


,Name,ProductSubcategoryID,ListPrice,ProductCategoryID
0,"HL Road Frame - Black, 58",14.0,1431.50,2
1,"HL Road Frame - Red, 58",14.0,1431.50,2
2,"Sport-100 Helmet, Red",31.0,34.99,4
3,"Sport-100 Helmet, Black",31.0,34.99,4
4,"Mountain Bike Socks, M",23.0,9.50,3


In [189]:
pc = db_reader().read_table('production.productcategory', connection)
pc.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [192]:
c = merge().merge_tbl(cat[['Name','ProductCategoryID', 'ListPrice']], pc[['ProductCategoryID', 'Name']], 'inner', 'ProductCategoryID').rename(columns={'Name_x' : 'p_name', 'Name_y' : 'c_name'})
c.head()

,p_name,ProductCategoryID,ListPrice,c_name
0,"HL Road Frame - Black, 58",2,1431.50,Components
1,"HL Road Frame - Red, 58",2,1431.50,Components
2,"Sport-100 Helmet, Red",4,34.99,Accessories
3,"Sport-100 Helmet, Black",4,34.99,Accessories
4,"Mountain Bike Socks, M",3,9.50,Clothing


In [193]:
# 247 Retrieve the list of orders along with the names of customers who placed those orders, matching on customer ID, and filtering orders placed in the last month.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [194]:
cus = db_reader().read_table('person.person', connection)
cus.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [195]:
name = merge().merge_tbl(soh[['SalesOrderID','CustomerID', 'OrderDate']], cus[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
name.head()

,SalesOrderID,CustomerID,OrderDate,BusinessEntityID,FirstName,LastName
0,43700,14501,2011-05-31,14501,Bryan,Gray
1,43701,11003,2011-05-31,11003,Jenna,Young
2,43703,16624,2011-06-01,16624,Nichole,Goel
3,43704,11005,2011-06-01,11005,Joe,Madan
4,43705,11011,2011-06-01,11011,Joe,Rana


In [197]:
name[name['OrderDate'] >= (max(name['OrderDate']) - pd.DateOffset(months = 1))].head()

,SalesOrderID,CustomerID,OrderDate,BusinessEntityID,FirstName,LastName
16780,74096,11877,2014-05-30,11877,Emma,Garcia
16781,74097,11371,2014-05-30,11371,Marshall,Xu
16782,74105,20223,2014-05-30,20223,George,Perez
16783,74106,16250,2014-05-30,16250,Samuel,Gonzalez
16784,74107,13570,2014-05-30,13570,Miguel,Foster


In [198]:
# 248 Get the total sales amount for each product category, along with the number of products in each category, only for categories with more than 10 products.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [199]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [200]:
cat= merge().merge_tbl(pro[['ProductID','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID','ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [202]:
cat.groupby('ProductCategoryID').agg({'ListPrice' : 'sum', 'ProductID' : 'count'}).rename(columns={'ListPrice' : 'Sum', 'ProductID' : 'Count'}).reset_index().query('Count > 10').head()

,ProductCategoryID,Sum,Count
0,1,14227.378,97
1,2,62961.280,134
2,3,1793.600,36
3,4,996.120,29


In [ ]:
# 249 Retrieve the total sales amount for each product category, along with the number of discontinued products in each category.

In [203]:
pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [204]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [205]:
cat= merge().merge_tbl(pro[['ProductID','ProductSubcategoryID', 'ListPrice', 'DiscontinuedDate']], psc[['ProductSubcategoryID','ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,ProductSubcategoryID,ListPrice,DiscontinuedDate,ProductCategoryID
0,680,14.0,1431.50,NaT,2
1,706,14.0,1431.50,NaT,2
2,707,31.0,34.99,NaT,4
3,708,31.0,34.99,NaT,4
4,709,23.0,9.50,NaT,3


In [206]:
cat.groupby('ProductCategoryID').agg({'ListPrice' : 'sum', 'DiscontinuedDate' : 'count'}).rename(columns={'ListPrice' : 'Sum', 'DiscontinuedDate' : 'Count'}).reset_index().head()

,ProductCategoryID,Sum,Count
0,1,14227.378,0
1,2,62961.280,0
2,3,1793.600,1
3,4,996.120,0


In [207]:
# 250 Retrieve the list of orders along with the names of customers who placed those orders and the shipping method used for each order.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [208]:
cus = db_reader().read_table('person.person', connection)
cus.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [209]:
name = merge().merge_tbl(soh[['SalesOrderID','CustomerID', 'ShipMethodID']], cus[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
name.head()

,SalesOrderID,CustomerID,ShipMethodID,BusinessEntityID,FirstName,LastName
0,43700,14501,1,14501,Bryan,Gray
1,43701,11003,1,11003,Jenna,Young
2,43703,16624,1,16624,Nichole,Goel
3,43704,11005,1,11005,Joe,Madan
4,43705,11011,1,11011,Joe,Rana


In [210]:
s = db_reader().read_table('Purchasing.ShipMethod', connection)
s.head()

,ShipMethodID,Name,ShipBase,ShipRate,rowguid,ModifiedDate
0,1,XRQ - TRUCK GROUND,3.95,0.99,6BE756D9-D7BE-4463-8F2C-AE60C710D606,2008-04-30
1,2,ZY - EXPRESS,9.95,1.99,3455079B-F773-4DC6-8F1E-2A58649C4AB8,2008-04-30
2,3,OVERSEAS - DELUXE,29.95,2.99,22F4E461-28CF-4ACE-A980-F686CF112EC8,2008-04-30
3,4,OVERNIGHT J-FAST,21.95,1.29,107E8356-E7A8-463D-B60C-079FFF467F3F,2008-04-30
4,5,CARGO TRANSPORT 5,8.99,1.49,B166019A-B134-4E76-B957-2B0490C610ED,2008-04-30


In [211]:
sn = merge().merge_tbl(name[['SalesOrderID', 'CustomerID', 'ShipMethodID', 'FirstName', 'LastName']], s[['ShipMethodID', 'Name']], 'inner', 'ShipMethodID')
sn.head()

,SalesOrderID,CustomerID,ShipMethodID,FirstName,LastName,Name
0,43700,14501,1,Bryan,Gray,XRQ - TRUCK GROUND
1,43701,11003,1,Jenna,Young,XRQ - TRUCK GROUND
2,43703,16624,1,Nichole,Goel,XRQ - TRUCK GROUND
3,43704,11005,1,Joe,Madan,XRQ - TRUCK GROUND
4,43705,11011,1,Joe,Rana,XRQ - TRUCK GROUND
